In [162]:
import os
import time
import pandas as pd
from alpha_vantage.timeseries import TimeSeries

In [163]:
API_KEY = 'Z0B7RAVHK9ASPW5O'

In [105]:
companies = {
    'RELIANCE.BSE': 'Reliance Industries',
    'TCS.BSE': 'Tata Consultancy Services',
    'HDFCBANK.BSE': 'HDFC Bank',
    'INFY.BSE': 'Infosys',
    'HINDUNILVR.BSE': 'Hindustan Unilever',
    'ICICIBANK.BSE': 'ICICI Bank',
    'KOTAKBANK.BSE': 'Kotak Mahindra Bank',
    'SBIN.BSE': 'State Bank of India',
    'BAJFINANCE.BSE': 'Bajaj Finance',
    'BHARTIARTL.BSE': 'Bharti Airtel'
}

start_date = '2020-01-01'
end_date = '2024-05-31'

# Initialize TimeSeries object
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')

In [106]:
def fetch_historical_data(symbol, start_date, end_date):
    try:
        data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
        data = data.rename({'index': 'date'}, axis=1)
        old_to_new_names = {
                    '1. open': 'open',
                    '2. high': 'high',
                    '3. low': 'low',
                    '4. close': 'close',
                    '5. volume': 'volume'
                }

        data = data.rename(columns=old_to_new_names)        
        data = data.loc[(data['date'] >= start_date) & (data['date'] <= end_date)]
        data['symbol'] = symbol
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

In [112]:
def save_data_to_sqlite(data):
    data.to_sql('stock_data', conn, if_exists='append')
    # Create indexes after data is saved
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON stock_data(date)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_symbol ON stock_data(symbol)')
    conn.commit()


In [113]:


# Fetch data for each company
all_data = []
for symbol, company in companies.items():
    print(f"Fetching data for {company} ({symbol})...")
    data = fetch_historical_data(symbol, start_date, end_date)
    if data is not None:
        all_data.append(data)
        save_data_to_sqlite(data)
        
    # Add a delay to respect the API rate limit
    time.sleep(12)  # 5 requests per minute = 12 seconds per request

Fetching data for Reliance Industries (RELIANCE.BSE)...
Fetching data for Tata Consultancy Services (TCS.BSE)...
Fetching data for HDFC Bank (HDFCBANK.BSE)...
Fetching data for Infosys (INFY.BSE)...
Fetching data for Hindustan Unilever (HINDUNILVR.BSE)...
Fetching data for ICICI Bank (ICICIBANK.BSE)...
Fetching data for Kotak Mahindra Bank (KOTAKBANK.BSE)...
Fetching data for State Bank of India (SBIN.BSE)...
Fetching data for Bajaj Finance (BAJFINANCE.BSE)...
Fetching data for Bharti Airtel (BHARTIARTL.BSE)...


In [114]:
if all_data:
    combined_data = pd.concat(all_data)
    # Calculate the size of the DataFrame in bytes
    size_in_bytes = combined_data.memory_usage(deep=True).sum()
    # Convert size to megabytes
    size_in_mb = size_in_bytes / (1024 ** 2)
    print(f"Size of the combined data: {size_in_mb:.2f} MB")
    combined_data.to_csv('data/historical_data.csv')
    print("Data saved to historical_data.csv")
else:
    print("No data fetched.")

Size of the combined data: 1.75 MB
Data saved to historical_data.csv


<h1>testing code</h1>

In [178]:
symbol='RELIANCE.BSE'

In [179]:
data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')

In [180]:
data['Company'] = symbol
data = data.rename({'index': 'date'}, axis=1)

In [181]:
old_to_new_names = {
    '1. open': 'open',
    '2. high': 'high',
    '3. low': 'low',
    '4. close': 'close',
    '5. volume': 'volume'
}

data = data.rename(columns=old_to_new_names)


In [182]:
data

,date,open,high,low,close,volume,Company
index,,,,,,,
0,2024-07-05,3114.6000,3197.6500,3097.0000,3188.0000,758770.0,RELIANCE.BSE
1,2024-07-04,3116.9500,3134.5000,3102.3000,3107.9000,132480.0,RELIANCE.BSE
2,2024-07-03,3130.3500,3149.5000,3085.2000,3105.3000,171680.0,RELIANCE.BSE
3,2024-07-02,3133.8000,3150.0000,3113.3500,3132.3000,157110.0,RELIANCE.BSE
4,2024-07-01,3129.9500,3157.5500,3110.4000,3120.3500,187610.0,RELIANCE.BSE
...,...,...,...,...,...,...,...
4803,2005-01-07,359.8269,373.1159,359.5202,368.5840,16969845.0,RELIANCE.BSE
4804,2005-01-06,357.7824,365.6195,356.1468,358.4297,13446517.0,RELIANCE.BSE
4805,2005-01-05,358.1230,363.9158,351.8194,361.1217,16954266.0,RELIANCE.BSE


In [183]:
def get_yesterday_date():
    from datetime import date, timedelta
    today = date.today()
    yesterday = today - timedelta(days = 1)
    return yesterday.strftime('%Y-%m-%d')
yesterday = get_yesterday_date()
yesterday

'2024-07-06'

In [132]:
data = data.loc[(data['date'] >= yesterday) & (data['date'] <= yesterday)]
data

,date,open,high,low,close,volume,Company
index,,,,,,,
0,2024-07-05,1421.85,1433.3,1417.0,1428.25,544720.0,BHARTIARTL.BSE


In [185]:
data['Company']

index
0       RELIANCE.BSE
1       RELIANCE.BSE
2       RELIANCE.BSE
3       RELIANCE.BSE
4       RELIANCE.BSE
            ...     
4803    RELIANCE.BSE
4804    RELIANCE.BSE
4805    RELIANCE.BSE
4806    RELIANCE.BSE
4807    RELIANCE.BSE
Name: Company, Length: 4808, dtype: object

In [37]:
data

,date,1. open,2. high,3. low,4. close,5. volume,symbol
index,,,,,,,
0,2024-07-05,3114.6000,3197.6500,3097.0000,3188.0000,758770.0,RELIANCE.BSE
1,2024-07-04,3116.9500,3134.5000,3102.3000,3107.9000,132480.0,RELIANCE.BSE
2,2024-07-03,3130.3500,3149.5000,3085.2000,3105.3000,171680.0,RELIANCE.BSE
3,2024-07-02,3133.8000,3150.0000,3113.3500,3132.3000,157110.0,RELIANCE.BSE
4,2024-07-01,3129.9500,3157.5500,3110.4000,3120.3500,187610.0,RELIANCE.BSE
...,...,...,...,...,...,...,...
4803,2005-01-07,359.8269,373.1159,359.5202,368.5840,16969845.0,RELIANCE.BSE
4804,2005-01-06,357.7824,365.6195,356.1468,358.4297,13446517.0,RELIANCE.BSE
4805,2005-01-05,358.1230,363.9158,351.8194,361.1217,16954266.0,RELIANCE.BSE


In [186]:
import sqlite3

In [187]:
conn = sqlite3.connect('central-stock-data.db')

In [68]:
if data is not None:
        save_data_to_sqlite(data)

In [ ]:
conn.close()

print("Data saved to historical_data.db")

In [148]:
symbol

'BHARTIARTL.BSE'

In [265]:
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')

data = ts.get_daily(symbol='RELIANCE.BSE', outputsize='full')[0]
old_to_new_names = {
            '1. open': 'open',
            '2. high': 'high',
            '3. low': 'low',
            '4. close': 'close',
            '5. volume': 'volume'
        }

data = data.rename(columns=old_to_new_names)      
data = data.rename({'index': 'date'}, axis=1)

data = data.loc[(data['date'] >= start_date) & (data['date'] <= end_date)]
data['Company'] = symbol
        # return data

ValueError: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.

In [189]:
data



,date,open,high,low,close,volume,Company
index,,,,,,,
24,2024-05-31,2864.6500,2884.2000,2843.2500,2859.6000,797290.0,RELIANCE.BSE
25,2024-05-30,2884.0000,2884.0000,2841.5000,2850.0000,54130.0,RELIANCE.BSE
26,2024-05-29,2897.0000,2916.1000,2877.0000,2881.4500,65080.0,RELIANCE.BSE
27,2024-05-28,2941.9500,2957.0000,2905.4000,2911.2500,116320.0,RELIANCE.BSE
28,2024-05-27,2960.8000,2967.4500,2924.0000,2932.4500,134570.0,RELIANCE.BSE
...,...,...,...,...,...,...,...
1113,2020-01-07,1385.6616,1398.4242,1380.6021,1390.2652,509038.0,RELIANCE.BSE
1114,2020-01-06,1392.0429,1392.9545,1365.1501,1368.7966,457443.0,RELIANCE.BSE
1115,2020-01-03,1398.7889,1405.0790,1388.7611,1401.3870,357456.0,RELIANCE.BSE


In [195]:
conn = sqlite3.connect('central_stock_data.db')
cursor = conn.cursor()

data.to_sql('historical_stock_data', conn, if_exists='append')
conn.commit()



In [227]:
query = '''
select company, date, close, lag(close,1) over (order by date) as prev_close from historical_stock_data




 '''

# query='select * from historical_stock_data'

df = pd.read_sql(query,conn)

# Execute the query
cursor.execute(query)

# Fetch all results as a list of tuples
df = cursor.fetchall()
df=pd.DataFrame(df)
# Close the connection
df

,0,1,2,3
0,RELIANCE.BSE,2020-01-01,1376.2719,NaN
1,RELIANCE.BSE,2020-01-01,1376.2719,1376.2719
2,RELIANCE.BSE,2020-01-01,1376.2719,1376.2719
3,RELIANCE.BSE,2020-01-02,1399.6549,1376.2719
4,RELIANCE.BSE,2020-01-02,1399.6549,1399.6549
...,...,...,...,...
3277,RELIANCE.BSE,2024-05-30,2850.0000,2850.0000
3278,RELIANCE.BSE,2024-05-30,2850.0000,2850.0000
3279,RELIANCE.BSE,2024-05-31,2859.6000,2850.0000
3280,RELIANCE.BSE,2024-05-31,2859.6000,2859.6000


In [234]:
data

,date,open,high,low,close,volume,Company
index,,,,,,,
24,2024-05-31,2864.6500,2884.2000,2843.2500,2859.6000,797290.0,RELIANCE.BSE
25,2024-05-30,2884.0000,2884.0000,2841.5000,2850.0000,54130.0,RELIANCE.BSE
26,2024-05-29,2897.0000,2916.1000,2877.0000,2881.4500,65080.0,RELIANCE.BSE
27,2024-05-28,2941.9500,2957.0000,2905.4000,2911.2500,116320.0,RELIANCE.BSE
28,2024-05-27,2960.8000,2967.4500,2924.0000,2932.4500,134570.0,RELIANCE.BSE
...,...,...,...,...,...,...,...
1113,2020-01-07,1385.6616,1398.4242,1380.6021,1390.2652,509038.0,RELIANCE.BSE
1114,2020-01-06,1392.0429,1392.9545,1365.1501,1368.7966,457443.0,RELIANCE.BSE
1115,2020-01-03,1398.7889,1405.0790,1388.7611,1401.3870,357456.0,RELIANCE.BSE


In [251]:
import mysql.connector

MYSQL_HOST = "stock-database.ctuyy6isws2v.ap-south-1.rds.amazonaws.com"
MYSQL_USER = "admin"
MYSQL_PASSWORD = "atlys-data"
MYSQL_DATABASE = ""

conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)
cursor = conn.cursor()

In [252]:
cursor.execute('CREATE DATABASE if not exists central_stock_db;')
cursor.execute('use central_stock_db;')

In [254]:


cursor.execute("""CREATE TABLE IF NOT EXISTS historical_stock_data (
        date DATE PRIMARY KEY,
        open DECIMAL(10,2) NOT NULL,
        high DECIMAL(10,2) NOT NULL,
        low DECIMAL(10,2) NOT NULL,
        close DECIMAL(10,2) NOT NULL,
        volume BIGINT NOT NULL,
        Company VARCHAR(255) NOT NULL
    )""")

    # Insert data using a bulk insert for efficiency
for index, row in data.iterrows():
    sql = """INSERT INTO historical_stock_data (date, open, high, low, close, volume, Company)
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    cursor.execute(sql, tuple(row.tolist()))

conn.commit()


In [262]:
query='''SELECT 
  h.Company,
  h.date,
  h.close,
  round(((h.close - lag(h.close, 1) OVER (PARTITION BY h.Company ORDER BY h.date)) / lag(h.close, 1) OVER (PARTITION BY h.Company ORDER BY h.date)) * 100,2) AS daily_variation
FROM historical_stock_data h
limit 10;
'''
cursor.execute(query)

In [263]:
for i in cursor:
    print(i)


('RELIANCE.BSE', datetime.date(2020, 1, 1), Decimal('1376.27'), None)
('RELIANCE.BSE', datetime.date(2020, 1, 2), Decimal('1399.65'), Decimal('1.70'))
('RELIANCE.BSE', datetime.date(2020, 1, 3), Decimal('1401.39'), Decimal('0.12'))
('RELIANCE.BSE', datetime.date(2020, 1, 6), Decimal('1368.80'), Decimal('-2.33'))
('RELIANCE.BSE', datetime.date(2020, 1, 7), Decimal('1390.27'), Decimal('1.57'))
('RELIANCE.BSE', datetime.date(2020, 1, 8), Decimal('1380.15'), Decimal('-0.73'))
('RELIANCE.BSE', datetime.date(2020, 1, 9), Decimal('1411.14'), Decimal('2.25'))
('RELIANCE.BSE', datetime.date(2020, 1, 10), Decimal('1410.91'), Decimal('-0.02'))
('RELIANCE.BSE', datetime.date(2020, 1, 13), Decimal('1407.04'), Decimal('-0.27'))
('RELIANCE.BSE', datetime.date(2020, 1, 14), Decimal('1393.91'), Decimal('-0.93'))


In [249]:
c


1094

In [264]:
cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON historical_stock_data(date)')


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'IF NOT EXISTS idx_date ON historical_stock_data(date)' at line 1